In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras import layers, callbacks
import tensorflow as tf
import keras
from tensorflow.keras.models import Sequential,Model
from keras.layers import *
from keras import backend as K
from tensorflow.keras.optimizers import Adam
from sklearn import metrics
from sklearn.metrics import accuracy_score, cohen_kappa_score, roc_auc_score, matthews_corrcoef, average_precision_score,confusion_matrix
from sklearn.metrics import classification_report
from sklearn import model_selection, metrics
from sklearn.model_selection import train_test_split,StratifiedKFold,KFold

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
HemoPI1_model = pd.read_csv('./HemoPI1.csv', index_col=0)

In [ ]:
m=0
clas=['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
encode=[]
ind=0
for i in range(len(HemoPI1_model)):
    st=HemoPI1_model['Sequence'][i][2:-1]
    row=[]
    if len(st)<=30:
      m=m+1
    for j in range(50):
        if j<len(st):
            index=clas.index(st[j])
            l=np.zeros(20)
            l[index]=1
        else:
            l=np.zeros(20)
        row.append(l)
    row=np.array(row)
    encode.append(row)
encode=np.array(encode)

In [ ]:
m/len(HemoPI1_model)

0.9166666666666666

In [ ]:
x_model1=encode
y_model1=HemoPI1_model['y_model_2cl']
x_train1, x_test1, y_train1, y_test1= train_test_split(x_model1,y_model1, test_size=0.2, random_state=0)
x_train1, x_val1, y_train1, y_val1= train_test_split(x_train1,y_train1, test_size=0.25, random_state=0)

In [ ]:
tf.random.set_seed(0)
np.random.seed(0)


In [ ]:
ls ='/content/drive/My Drive/best_model/'

In [ ]:

scores =[]
i=1
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),tf.keras.callbacks.ModelCheckpoint(filepath=ls+'best_model1.h5', monitor='val_accuracy', save_best_only=True,mode='auto')]

model1 = Sequential()
model1.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same',input_shape=(50,20,1)))
model1.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same'))
model1.add(Conv2D(64, kernel_size=(7,7),activation='relu',padding='same'))
model1.add(Conv2D(64, kernel_size=(5,5),activation='relu',padding='same'))
model1.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model1.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model1.add(Flatten())
model1.add(tf.keras.layers.Dense(64, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(l2=0.0001)))
model1.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid,kernel_regularizer=tf.keras.regularizers.l2(l2=0.1)))
optimizer = Adam(learning_rate=0.0001)
model1.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model1.fit(x_train1,y_train1,epochs = 100,batch_size=32,validation_data=(x_val1,y_val1),callbacks=[callback],verbose=0,shuffle= True)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model1 = tf.keras.models.load_model(ls+'HemoPI1.h5')
y_pred=model1.predict(x_model1)
y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0

cv_preds = y_pred
print('HemoPI-1 model datasets')
name='deep learning'
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_model1, cv_preds)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_model1, cv_preds)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_model1, cv_preds)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_model1, cv_preds)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_model1, cv_preds)))
target_names = ['low 0', 'high 1']
print(classification_report(y_model1, cv_preds, target_names=target_names))

# Predictions Validation Set
print('HemoPI-1 validation datasets')
y_pred2=model1.predict(x_test1)
y_pred2[y_pred2>0.5]=1
y_pred2[y_pred2<0.5]=0
cv_preds2 = y_pred2
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_test1, cv_preds2)))
print("%s: Precision %0.2f%%" % (name, 100*metrics.precision_score(y_test1, cv_preds2)))
print("%s: Recall %0.2f%%" % (name, 100*metrics.recall_score(y_test1, cv_preds2)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_test1, cv_preds2)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_test1, cv_preds2)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_test1, cv_preds2)))
target_names = ['low 0', 'high 1']
print(classification_report(y_test1, cv_preds2, target_names=target_names))

35/35 [==============================] - 0s 6ms/step
HemoPI-1 model datasets
deep learning: Accuracy 97.37%
deep learning: Precision-Recall 95.54%
deep learning: Matthews Coefficient 94.78%
deep learning: Cohen Kappa Score 94.75%
deep learning: ROC AUC Score 97.37%
              precision    recall  f1-score   support

       low 0       0.99      0.96      0.97       552
      high 1       0.96      0.99      0.97       552

    accuracy                           0.97      1104
   macro avg       0.97      0.97      0.97      1104
weighted avg       0.97      0.97      0.97      1104

HemoPI-1 validation datasets
7/7 [==============================] - 0s 6ms/step
deep learning: Accuracy 96.38%
deep learning: Precision 95.19%
deep learning: Recall 97.06%
deep learning: Matthews Coefficient 92.74%
deep learning: Cohen Kappa Score 92.73%
deep learning: ROC AUC Score 96.43%
              precision    recall  f1-score   support

       low 0       0.97      0.96      0.97       119
      h

In [ ]:
HemoPI2_model = pd.read_csv('./HemoPI2.csv', index_col=0)

In [ ]:
clas=['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']

encode2=[]
ind=0
m=0
for i in range(len(HemoPI2_model)):
    st=HemoPI2_model['Sequence'][i][2:-1]
    row=[]
    for j in range(50):
        if j<len(st):
            index=clas.index(st[j])
            l=np.zeros(20)
            l[index]=1
        else:
            l=np.zeros(20)
        row.append(l)
    row=np.array(row)
    encode2.append(row)
encode2=np.array(encode2)


In [ ]:
x_model2=encode2
y_model2=HemoPI2_model['y_model_2cl']
x_train2, x_test2, y_train2, y_test2= train_test_split(x_model2,y_model2, test_size=0.2, random_state=0)
x_train2, x_val2, y_train2, y_val2 = train_test_split(x_train2,y_train2, test_size=0.25, random_state=0)


In [ ]:
scores =[]
i=1
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),tf.keras.callbacks.ModelCheckpoint(filepath=ls+'best_model2.h5', monitor='val_accuracy',save_best_only=True,mode='auto')]

model2 = Sequential()
model2.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same',input_shape=(50,20,1)))
model2.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same'))
model2.add(Conv2D(64, kernel_size=(7,7),activation='relu',padding='same'))
model2.add(Conv2D(64, kernel_size=(5,5),activation='relu',padding='same'))
model2.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model2.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model2.add(Flatten())
model2.add(tf.keras.layers.Dense(64, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(l2=0.0001)))
model2.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid,kernel_regularizer=tf.keras.regularizers.l2(l2=0.01)))
optimizer = Adam(learning_rate=0.0001)
model2.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model2.fit(x_train2,y_train2,epochs = 100,batch_size=32,validation_data=(x_val2,y_val2),callbacks=[callback],verbose=0,shuffle= True)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model2 = tf.keras.models.load_model(ls+'HemoPI2.h5')
y_pred=model2.predict(x_model2)
y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0

cv_preds = y_pred
print('HemoPI-2 model datasets')
name='deep learning'
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_model2, cv_preds)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_model2, cv_preds)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_model2, cv_preds)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_model2, cv_preds)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_model2, cv_preds)))
tn, fp, fn, tp = confusion_matrix(y_model2, cv_preds).ravel()

# calculate sensitivity
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)
specificity = tn / (tn + fp)
print("Specificity:", specificity)

target_names = ['low 0', 'high 1']
print(classification_report(y_model2, cv_preds, target_names=target_names))

# Predictions Validation Set
print('HemoPI-2 validation datasets')
y_pred2=model2.predict(x_test2)
y_pred2[y_pred2>0.5]=1
y_pred2[y_pred2<0.5]=0
cv_preds2 = y_pred2
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_test2, cv_preds2)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.precision_score(y_test2, cv_preds2)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_test2, cv_preds2)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_test2, cv_preds2)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_test2, cv_preds2)))
tn, fp, fn, tp = confusion_matrix(y_test2, cv_preds2).ravel()

# calculate sensitivity
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)
specificity = tn / (tn + fp)
print("Specificity:", specificity)

target_names = ['low 0', 'high 1']
print(classification_report(y_test2, cv_preds2, target_names=target_names))

32/32 [==============================] - 0s 7ms/step
HemoPI-2 model datasets
deep learning: Accuracy 87.38%
deep learning: Precision-Recall 83.04%
deep learning: Matthews Coefficient 74.70%
deep learning: Cohen Kappa Score 74.33%
deep learning: ROC AUC Score 86.84%
Sensitivity: 0.9293478260869565
Specificity: 0.8073593073593074
              precision    recall  f1-score   support

       low 0       0.91      0.81      0.85       462
      high 1       0.85      0.93      0.89       552

    accuracy                           0.87      1014
   macro avg       0.88      0.87      0.87      1014
weighted avg       0.88      0.87      0.87      1014

HemoPI-2 validation datasets
7/7 [==============================] - 0s 6ms/step
deep learning: Accuracy 77.83%
deep learning: Precision-Recall 74.42%
deep learning: Matthews Coefficient 56.14%
deep learning: Cohen Kappa Score 54.88%
deep learning: ROC AUC Score 77.08%
Sensitivity: 0.8888888888888888
Specificity: 0.6526315789473685
          

In [ ]:
HemoPI3_model = pd.read_csv('./HemoPI3.csv', index_col=0)

In [ ]:
clas=['A','C','D','E','F','G','H','I','K','L','M','N','P','Q','R','S','T','V','W','Y']
encode3=[]
for i in range(len(HemoPI3_model)):
    st=HemoPI3_model['Sequence'][i][2:-1]
    row=[]
    for j in range(50):
        if j<len(st):
            index=clas.index(st[j])
            l=np.zeros(20)
            l[index]=1
        else:
            l=np.zeros(20)
        row.append(l)
    row=np.array(row)
    encode3.append(row)
encode3=np.array(encode3)


In [ ]:
x_model3=encode3
y_model3=HemoPI3_model['y_model_2cl']
x_train3, x_test3, y_train3, y_test3 = train_test_split(x_model3,y_model3, test_size=0.2, random_state=0)
x_train3, x_val3, y_train3, y_val3 = train_test_split(x_train3,y_train3, test_size=0.25, random_state=0)


In [ ]:
i=1
callback = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10),tf.keras.callbacks.ModelCheckpoint(filepath=ls+'best_model3.h5', monitor='val_accuracy', save_best_only=True,mode='auto')]

model3 = Sequential()
model3.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same',input_shape=(50,20,1)))
model3.add(Conv2D(32, kernel_size=(11, 11),activation='relu',padding='same'))
model3.add(Conv2D(64, kernel_size=(7,7),activation='relu',padding='same'))
model3.add(Conv2D(64, kernel_size=(5,5),activation='relu',padding='same'))
model3.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model3.add(Conv2D(128, kernel_size=(3,3),activation='relu',padding='same'))
model3.add(Flatten())
model3.add(tf.keras.layers.Dense(64, activation=tf.nn.relu,kernel_regularizer=tf.keras.regularizers.l2(l2=0.001)))
model3.add(tf.keras.layers.Dense(1, activation=tf.nn.sigmoid,kernel_regularizer=tf.keras.regularizers.l2(l2=0.1)))
optimizer = Adam(learning_rate=0.0001)
model3.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
history = model3.fit(x_train3,y_train3,epochs = 100,batch_size=32,validation_data=(x_val3,y_val3),callbacks=[callback],verbose=0,shuffle=True)


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
model3 = tf.keras.models.load_model(ls+'HemoPI3.h5')
y_pred=model3.predict(x_model3)
y_pred[y_pred>0.5]=1
y_pred[y_pred<0.5]=0

cv_preds = y_pred
print('HemoPI-3 model datasets')
name='deep learning'
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_model3, cv_preds)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_model3, cv_preds)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_model3, cv_preds)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_model3, cv_preds)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_model3, cv_preds)))
tn, fp, fn, tp = confusion_matrix(y_model3, cv_preds).ravel()

# calculate sensitivity
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)
specificity = tn / (tn + fp)
print("Specificity:", specificity)

target_names = ['low 0', 'high 1']
print(classification_report(y_model3, cv_preds, target_names=target_names))

# Predictions Validation Set
print('HemoPI-3 validation datasets')
y_pred2=model3.predict(x_test3)
y_pred2[y_pred2>0.5]=1
y_pred2[y_pred2<0.5]=0
cv_preds2 = y_pred2
print("%s: Accuracy %0.2f%%" % (name, 100*metrics.accuracy_score(y_test3, cv_preds2)))
print("%s: Precision-Recall %0.2f%%" % (name, 100*metrics.average_precision_score(y_test3, cv_preds2)))
print("%s: Matthews Coefficient %0.2f%%" % (name, 100*matthews_corrcoef(y_test3, cv_preds2)))
print("%s: Cohen Kappa Score %0.2f%%" % (name, 100*metrics.cohen_kappa_score(y_test3, cv_preds2)))
print("%s: ROC AUC Score %0.2f%%" % (name, 100*metrics.roc_auc_score(y_test3, cv_preds2)))
tn, fp, fn, tp = confusion_matrix(y_test3, cv_preds2).ravel()

# calculate sensitivity
sensitivity = tp / (tp + fn)
print("Sensitivity:", sensitivity)
specificity = tn / (tn + fp)
print("Specificity:", specificity)

target_names = ['low 0', 'high 1']
print(classification_report(y_test3, cv_preds2, target_names=target_names))

51/51 [==============================] - 0s 5ms/step
HemoPI-3 model datasets
deep learning: Accuracy 88.97%
deep learning: Precision-Recall 86.09%
deep learning: Matthews Coefficient 77.74%
deep learning: Cohen Kappa Score 77.73%
deep learning: ROC AUC Score 88.82%
Sensitivity: 0.9050847457627119
Specificity: 0.8712737127371274
              precision    recall  f1-score   support

       low 0       0.88      0.87      0.88       738
      high 1       0.89      0.91      0.90       885

    accuracy                           0.89      1623
   macro avg       0.89      0.89      0.89      1623
weighted avg       0.89      0.89      0.89      1623

HemoPI-3 validation datasets
11/11 [==============================] - 0s 5ms/step
deep learning: Accuracy 80.31%
deep learning: Precision-Recall 75.37%
deep learning: Matthews Coefficient 60.51%
deep learning: Cohen Kappa Score 60.46%
deep learning: ROC AUC Score 80.18%
Sensitivity: 0.8294117647058824
Specificity: 0.7741935483870968
        